In [45]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [113]:
d = {'persona':['javi','carlos','angel','felipe','fernando'],'ES':[1,1,0,1,1],'FR':[1,0,1,1,0],'AL':[1,1,1,0,1],'BE':[0,1,1,0,1]}
d

{'AL': [1, 1, 1, 0, 1],
 'BE': [0, 1, 1, 0, 1],
 'ES': [1, 1, 0, 1, 1],
 'FR': [1, 0, 1, 1, 0],
 'persona': ['javi', 'carlos', 'angel', 'felipe', 'fernando']}

In [114]:
df = pd.DataFrame(data = d)

In [115]:
df

,AL,BE,ES,FR,persona
0,1,0,1,1,javi
1,1,1,1,0,carlos
2,1,1,0,1,angel
3,0,0,1,1,felipe
4,1,1,1,0,fernando


In [116]:
destinos = df.columns[0:4].tolist()
agrupados = df.groupby(destinos).count().reset_index()
agrupados

,AL,BE,ES,FR,persona
0,0,0,1,1,1
1,1,0,1,1,1
2,1,1,0,1,1
3,1,1,1,0,2


In [117]:
X= agrupados[destinos].values
X

array([[0, 0, 1, 1],
       [1, 0, 1, 1],
       [1, 1, 0, 1],
       [1, 1, 1, 0]])

In [118]:
vecinos = NearestNeighbors(n_neighbors=2,metric='cosine',algorithm='brute')
vecinos = vecinos.fit(X)

In [119]:
predecir = np.array([[1,0, 1, 0]])
distances, indices = vecinos.kneighbors(predecir)
agrupados.iloc[indices[0],:]

,AL,BE,ES,FR,persona
1,1,0,1,1,1
3,1,1,1,0,2


In [125]:
distances

array([[ 0.18350342,  0.18350342]])

In [120]:
candidatos = agrupados.iloc[indices[0],:][destinos].values
pesos = agrupados.iloc[indices[0],:]['persona'].values

In [121]:
def restar(a,b):
    lon = len(a)
    sol = [0]*lon
    for i in range(lon):
        sol[i] = a[i]-b[i]
    return sol

In [122]:
def suma(a,b):
    lon = len(a)
    sol = [0]*lon
    for i in range(lon):
        sol[i] = a[i]+b[i]
    return sol

In [123]:
def dar_destino(listaCandidatos,pesos,distances,predecir):
    destinos = []
    for i in listaCandidatos:
        aux = restar(i,predecir)
        destinos.append(aux) 
    destinoPesos = []
    for i in range(len(pesos)):
        distancia = distances[i]
        peso = pesos[i]
        destino = destinos[i]
        destinoPesos.append(map(lambda x: x*peso,destino))
    sol = reduce(lambda x,y:suma(x,y),destinoPesos,[0]*len(destinoPesos[0]))
    destino = sol.index(max(sol))
    return destino
        

In [124]:
destino = dar_destino(candidatos,pesos,distances[0],predecir[0])
destinos[destino]

'BE'